 # Food Classification Using Deep Neural Networks and Transfer Learning

# 1. Data Splitting 

Split the data into training, validation, and test sets. Justify your choice.

In [1]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms

import matplotlib.pyplot as plt


# import shutil
import os
import random
random.seed = 2019
split = [600,800,1000]

for food in ['falafel','apple_pie','donuts','french_fries','macarons','nachos','onion_rings','oysters','pizza', 'mussels','chicken_wings','chocolate_cake','fried_rice','greek_salad','hamburger','lasagna','poutine','spring_rolls','waffles','spaghetti_carbonara']:
    
    #create new empty folders
    if not os.path.exists('train_more_food/' + food):
        os.makedirs('train_more_food/' + food)
    if not os.path.exists('val_more_food/' + food):
        os.makedirs('val_more_food/' + food)
    if not os.path.exists('test_more_food/' + food):
        os.makedirs('test_more_food/' + food)
    
    #make a list of image file names, shuffle them
    pictures = os.listdir("More Food/"+ food)
    pictures.sort()
    random.shuffle(pictures)
    
    #copy images into folders
    for pic in pictures[0:600]:
        shutil.copy("More Food/"+ food +"/"+ pic,"train_more_food/"+food+"/"+pic)
        print("copy","More Food/"+ food +"/"+ pic,"to","train_more_food/"+food+"/"+pic)
        
    for pic in pictures[600:800]:
        shutil.copy("More Food/"+ food +"/"+ pic,"val_more_food/"+food+"/"+pic)
        print("copy","More Food/"+ food +"/"+ pic,"to","val_more_food/"+food+"/"+pic)
        
    for pic in pictures[800:1000]:
        shutil.copy("More Food/"+ food +"/"+ pic,"test_more_food/"+food+"/"+pic)
        print("copy","More Food/"+ food +"/"+ pic,"to","test_more_food/"+food+"/"+pic)

# Load Data

In [2]:
def get_data_loader(target_classes, batch_size):
    """ Returns the indices for datapoints in the dataset that
    belongs to the desired target classes, a subset of all possible classes.

    Args:
        dataset: Dataset object
        classes: A list of strings denoting the name of each class
        target_classes: A list of strings denoting the name of the desired
                        classes. Should be a subset of the argument 'classes'
    Returns:
        indices: list of indices that have labels corresponding to one of the
                 target classes
    """
    classes = ('falafel','apple_pie','donuts','french_fries','macarons','nachos','onion_rings','oysters','pizza', 'mussels')


    transform = transforms.Compose(
    [transforms.Resize((224,224), interpolation=2), transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    trainset = torchvision.datasets.ImageFolder(root = 'train',
                                                 transform=transform)
 
    train_loader = torch.utils.data.DataLoader(trainset, batch_size = batch_size,
                                              num_workers = 1, shuffle = True)

    
    val_set = torchvision.datasets.ImageFolder(root='val',
                                                 transform=transform)
    val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size,
                                              num_workers=1, shuffle = True)

    testset = torchvision.datasets.ImageFolder(root= 'test',
                                            transform=transform)
    
    test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                             num_workers=1, shuffle = True)

    return train_loader, classes#, val_loader, test_loader, classes

In [3]:
classes = ('falafel','apple_pie','donuts','french_fries','macarons','nachos','onion_rings','oysters','pizza', 'mussels')

transform = transforms.Compose(
    [transforms.Resize((224,224), interpolation=2), transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

trainset = torchvision.datasets.ImageFolder(root = 'train',
                                             transform=transform)

train_loader = torch.utils.data.DataLoader(trainset, batch_size = 64,
                                          num_workers = 1, shuffle = True)
val_set = torchvision.datasets.ImageFolder(root='val',
                                                 transform=transform)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=64,
                                              num_workers=1, shuffle = True)

testset = torchvision.datasets.ImageFolder(root= 'test',
                                            transform=transform)
    
test_loader = torch.utils.data.DataLoader(testset, batch_size=64,
                                             num_workers=1, shuffle = True)

In [4]:
print(len(trainset))

6000


In [ ]:
augmented_train_data = []

my_transform = transforms.Compose([
    transforms.RandomRotation(25),
    transforms.ToTensor(),
])

for i in range(2):
    mnist_new = torchvision.datasets.ImageFolder(root = 'train',
                                             transform=my_transform)
    for j, item in enumerate(mnist_new):
        augmented_train_data.append(item)

In [11]:
print(len(mnist_new))

6000


In [ ]:
print(len(trainset))

In [ ]:
print(len((trainset)))

In [28]:
trainset.class_to_idx


{'apple_pie': 0,
 'donuts': 1,
 'falafel': 2,
 'french_fries': 3,
 'macarons': 4,
 'mussels': 5,
 'nachos': 6,
 'onion_rings': 7,
 'oysters': 8,
 'pizza': 9}

In [29]:
train_loader_iter = iter(train_loader)
imgs, labels = next(train_loader_iter)
print(imgs.shape)
print(labels)
print(labels.shape)

torch.Size([64, 3, 224, 224])
tensor([0, 1, 1, 6, 8, 6, 4, 5, 7, 2, 0, 1, 0, 9, 6, 7, 1, 3, 1, 0, 8, 7, 6, 9,
        3, 9, 3, 9, 0, 5, 3, 0, 8, 1, 1, 3, 4, 1, 4, 0, 1, 7, 0, 2, 6, 6, 7, 2,
        9, 7, 5, 7, 6, 0, 7, 4, 8, 3, 3, 9, 8, 7, 8, 0])
torch.Size([64])


# 2. Convolutional Network - Baseline Model 
Build a convolutional neural network model that takes the (224x224 RGB) image as input.

In [30]:
import torch.nn as nn
import torch.nn.functional as F
 
class LargeNet(nn.Module):
    def __init__(self):
        super(LargeNet, self).__init__()
        self.name = "large"
        self.conv1 = nn.Conv2d(3, 32, 3, padding = 1) 
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding = 1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding = 1)
        
        self.fc1 = nn.Linear(128 * 28 * 28, 128)
        self.fc2 = nn.Linear(128, 9)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 28 * 28)  
        x = F.relu(self.fc1(x))
        x = self.fc2(x) 
        x = x.squeeze(1) # Flatten to [batch_size]
        return x

# 3. Training 
Train your network. Plot the training curve.

Make sure that you are checkpointing frequently!

In [31]:
import torch.nn as nn
import torch.nn.functional as F
# convolutional neural network, 
class LargeNet(nn.Module):
    def __init__(self):
        super(LargeNet, self).__init__()
        self.name = "large"
        self.conv1 = nn.Conv2d(3, 5, 3, padding = 1) 
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(5, 10, 3, padding = 1)
        self.conv3 = nn.Conv2d(10, 15, 3, padding = 1)
        self.fc1 = nn.Linear(15 * 28 * 28 , 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        
        x = x.view(-1, 15 * 28 * 28)  
        x = F.relu(self.fc1(x))
        x = self.fc2(x) 
        x = x.squeeze(1) # Flatten to [batch_size]
        return x

In [32]:
def get_model_name(name, batch_size, learning_rate, epoch):
    """ Generate a name for the model consisting of all the hyperparameter values

    Args:
        config: Configuration object containing the hyperparameters
    Returns:
        path: A string with the hyperparameter name and value concatenated
    """
    path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(name,
                                                   batch_size,
                                                   learning_rate,
                                                   epoch)
    return path


In [33]:
train_loader_iter = iter(train_loader)
imgs, labels = next(train_loader_iter)
print(imgs.shape)
print(labels)
print(labels.shape)

torch.Size([64, 3, 224, 224])
tensor([3, 5, 3, 7, 3, 6, 0, 9, 8, 9, 0, 8, 3, 6, 9, 9, 6, 3, 4, 8, 5, 3, 9, 8,
        0, 6, 1, 4, 0, 5, 1, 5, 4, 7, 4, 0, 3, 0, 5, 5, 9, 3, 5, 8, 8, 6, 4, 9,
        3, 2, 1, 0, 3, 2, 6, 1, 3, 1, 0, 3, 7, 0, 6, 9])
torch.Size([64])


# Training - Baseline Model

In [34]:
def evaluate(net, loader, criterion):
    """ Evaluate the network on the validation set.

     Args:
         net: PyTorch neural network object
         loader: PyTorch data loader for the validation set
         criterion: The loss function
     Returns:
         err: A scalar for the avg classification error over the validation set
         loss: A scalar for the average loss function over the validation set
     """
    total_loss = 0.0
    total_err = 0.0
    total_epoch = 0

    for i, data in enumerate(loader, 0):
        inputs, labels = data
        
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        pred = outputs.max(1, keepdim=True)[1] # get the index of the max log-probability
        total_err += pred.ne(labels.view_as(pred)).sum().item()
        total_loss += loss.item()
        total_epoch += len(labels)

    err = float(total_err) / total_epoch
    loss = float(total_loss) / (i + 1)

    return err, loss

In [35]:
def train(net, batch_size=1, learning_rate=0.0001, num_epochs=30):
    transform = transforms.Compose(
        [transforms.Resize((224,224)), transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    trainset = torchvision.datasets.ImageFolder(root='train',
                                                 transform=transform)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          num_workers=1, shuffle = True)
    
    train_loader_iter = iter(train_loader)
    imgs, labels = next(train_loader_iter)
    
    
    torch.manual_seed(1000)

   
  
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay=1e-5)
    
    train_err = np.zeros(num_epochs)
    train_loss = np.zeros(num_epochs)
    val_err = np.zeros(num_epochs)
    val_loss = np.zeros(num_epochs)
    
    
    iters, losses, train_acc, val_acc = [], [], [], []
    start_time = time.time()
    # training
    n=0
    for epoch in range(num_epochs):  # loop over the dataset multiple times 
      
        total_train_loss = 0.0
        total_train_err = 0.0

        total_epoch = 0
        for i, data in enumerate(train_loader, 0):
            
            inputs, labels = data
                      
            optimizer.zero_grad()
            outputs = net(inputs)
           
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            iters.append(n)
           
            pred = outputs.max(1, keepdim=True)[1] # get the index of the max log-probability
            total_train_err += pred.ne(labels.view_as(pred)).sum().item()
            total_train_loss += loss.item()
            total_epoch += len(labels)
           
            losses.append(float(loss)/batch_size)  
            n += 1# compute *average* loss
        
        train_err[epoch] = float(total_train_err) / total_epoch
     
        train_loss[epoch] = float(total_train_loss) / (i+1)
        
        val_err[epoch], val_loss[epoch] = evaluate(net, val_loader, criterion)
    
        print(("Epoch {}: Train err: {}, Train loss: {}  |" +
               "Validation err: {} , Validation loss:{} ").format(
                   epoch + 1,
            
                   train_err[epoch],
                   train_loss[epoch],
                   val_err[epoch],
                   val_loss[epoch]
                        ))

        # Save the current model (checkpoint) to a file
        model_path = get_model_name(net.name, batch_size, learning_rate, epoch)
        torch.save(net.state_dict(), model_path)

    print('Finished Training')
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Total time elapsed: {:.2f} seconds".format(elapsed_time))

    # Write the train/test loss/err into CSV file for plotting later
    epochs = np.arange(1, num_epochs + 1)

    np.savetxt("{}_train_err.csv".format(model_path), train_err)
    np.savetxt("{}_train_loss.csv".format(model_path), train_loss)
    np.savetxt("{}_val_err.csv".format(model_path), val_err)
    np.savetxt("{}_val_loss.csv".format(model_path), val_loss)
        
          

In [36]:
# Training Curve
import matplotlib.pyplot as plt
def plot_training_curve(path):
    """ Plots the training curve for a model run, given the csv files
    containing the train/validation error/loss.

    Args:
        path: The base path of the csv files produced during training
    """

    train_err = np.loadtxt("{}_train_err.csv".format(path))
    val_err = np.loadtxt("{}_val_err.csv".format(path))
    train_loss = np.loadtxt("{}_train_loss.csv".format(path))
    val_loss = np.loadtxt("{}_val_loss.csv".format(path))
    plt.title("Train vs Validation Error")
    plt.plot(range(1,16), train_err, label="Train")
    plt.plot(range(1,16), val_err, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Error")
    plt.legend(loc='best')
    plt.show()
    plt.title("Train vs Validation Loss")
    plt.plot(range(1,16), train_loss, label="Train")
    plt.plot(range(1,16), val_loss, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend(loc='best')
    plt.show()

# Hyperparameter Seach - Baseline Model

In [13]:
ne3 = LargeNet()

In [28]:
train(ne3, batch_size = 64, learning_rate=0.01, num_epochs = 15)

Epoch 1: Train err: 0.8581666666666666, Train loss: 2.292994164405985  |Validation err: 0.8265 , Validation loss:2.2168687283992767 
Epoch 2: Train err: 0.8133333333333334, Train loss: 2.1898040238847125  |Validation err: 0.8125 , Validation loss:2.1898394525051117 
Epoch 3: Train err: 0.7908333333333334, Train loss: 2.150301992893219  |Validation err: 0.8325 , Validation loss:2.231725499033928 
Epoch 4: Train err: 0.7218333333333333, Train loss: 2.024061899235908  |Validation err: 0.7735 , Validation loss:2.1689374707639217 
Epoch 5: Train err: 0.6418333333333334, Train loss: 1.8138254690677562  |Validation err: 0.7865 , Validation loss:2.2966490015387535 
Epoch 6: Train err: 0.5516666666666666, Train loss: 1.5930405474723655  |Validation err: 0.806 , Validation loss:2.587955057621002 


KeyboardInterrupt: 

In [ ]:
ne3 = LargeNet()
train(ne3, batch_size = 64, learning_rate=0.1, num_epochs = 15)

In [ ]:
ne3 = LargeNet()
train(ne3, batch_size = 32, learning_rate=0.001, num_epochs = 15)

In [ ]:
ne3 = LargeNet()
train(ne3, batch_size = 32, learning_rate=0.01, num_epochs = 15)

In [ ]:
ne3 = LargeNet()
train(ne3, batch_size = 32, learning_rate=0.1, num_epochs = 15)

In [ ]:
path = 'model_large_bs64_lr0.001_epoch14'
plot_training_curve(path)


# Test Classification Error

In [41]:
net = LargeNet()
model_path = get_model_name(net.name, batch_size=64, learning_rate=0.001, epoch=5)
state = torch.load(model_path)
net.load_state_dict(state)

evaluate(net, test_loader, nn.CrossEntropyLoss())


(0.581625, 1.6759631481170654)

In [42]:
The test classification error was about 0.37.

SyntaxError: invalid syntax (<ipython-input-42-3ddd3359581b>, line 1)

In [44]:
len(testset)

8000

# Test Accuracy

In [ ]:
def get_accuracy(model):
    data = testset
    total = 0
    correct = 0
    for i, data in enumerate(test_loader, 0):
        inputs, labels = data
        outputs = net(inputs)
        # We don't need to run F.softmax
        pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += imgs.shape[0]
    return correct / total

In [34]:
net = LargeNet()
model_path = get_model_name(net.name, batch_size=64, learning_rate=0.001, epoch=5)
state = torch.load(model_path)
net.load_state_dict(state)




In [35]:
get_accuracy(net)

TypeError: conv2d(): argument 'input' (position 1) must be Tensor, not int

# 5. Transfer Learning [16 pt]
For many image classification tasks, it is generally not a good idea to train a very large Deep Neural Network model from scratch due to the enormous compute requirements and lack of sufficient amounts of training data. One of the better option is to try using an existing model that performs a similar task to the one you need to solve. This method of utilizing a pre-trained network for other similar tasks is broadly termed â€œTransfer Learningâ€. In this assignment, we will use Transfer Learning to extract features from the hand gesture images. Then, train a smaller network to use these features as input and classify the hand gestures.

As you have learned from the CNN lecture, Convolution layers extract various features from the images which get utilized by the fully connected layers for correct classification. AlexNet architecture played a pivotal role in establishing Deep Neural Nets as a go-to tool for Image classification problems and we will use an imagenet pre-trained AlexNet model to extract features in this assignment.

In [16]:
from torchvision import transforms

# Set up a transform that scales and crops an image so it has the dimensions
# of the input layer of alexnet
scale_crop = transforms.Compose([
   transforms.Resize(256),
   transforms.CenterCrop(224)
])

# The normalization that was applied to the data when alexnet was trained
normalize = transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)

# Put scaling, cropping, normalzing and converting from PIL image to pytorch into one package
preprocess = transforms.Compose([
   scale_crop,
   transforms.ToTensor(),
   normalize
])

In [21]:
trainset = torchvision.datasets.ImageFolder(root = 'train',
                                             transform=preprocess)

train_loader = torch.utils.data.DataLoader(trainset, batch_size = 32,
                                          num_workers = 1, shuffle = True)
val_set = torchvision.datasets.ImageFolder(root='val',
                                                 transform=preprocess)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=32,
                                              num_workers=1, shuffle = True)

testset = torchvision.datasets.ImageFolder(root= 'test',
                                           transform=transform)
    
test_loader = torch.utils.data.DataLoader(testset, batch_size=32,
                                             num_workers=1, shuffle = True)

In [42]:
# Importing relevant Libraries
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import time
import os
import torchvision
import matplotlib.pyplot as plt

from torchvision import datasets, models, transforms
from torch.nn import functional as F
import copy
from torch.autograd import Variable


###############################################################################
# Feature Extraction using AlexNet pretrained model

class AlexNet(nn.Module):
    '''
    Class that loads AlexNet Feature Model ('Convolution layers') with imagenet trained weights
    
    input : image tensors with dimension Lx3x224x224
    
    output : feature tensor with dimension Lx256x6x6
    
    *L - Batch size
    
    '''
    
    def load_weights(self):
        an_builtin = torchvision.models.alexnet(pretrained=True) # Loads the pretrained model weights
        
        features_weight_i = [0, 3, 6, 8, 10]
        for i in features_weight_i:
            self.features[i].weight = an_builtin.features[i].weight
            self.features[i].bias = an_builtin.features[i].bias

    def __init__(self):
        super(AlexNet, self).__init__()
        self.name = "AlexNet"
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 10),
        )
        self.load_weights() # Copies the weights to AlexNetFeatures model layers

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x
        


In [44]:
netalex = AlexNet()

In [47]:
train(netalex, batch_size = 32, learning_rate = 0.0001, num_epochs = 15)

Epoch 1: Train err: 0.3893333333333333, Train loss: 1.1690362076810066  |Validation err: 0.25 , Validation loss:0.7569273803383112 
Epoch 2: Train err: 0.18666666666666668, Train loss: 0.5706664022613079  |Validation err: 0.245 , Validation loss:0.7428885139524937 
Epoch 3: Train err: 0.11716666666666667, Train loss: 0.3517896554412994  |Validation err: 0.216 , Validation loss:0.6798681542277336 
Epoch 4: Train err: 0.07, Train loss: 0.20826976659133078  |Validation err: 0.213 , Validation loss:0.6846165582537651 
Epoch 5: Train err: 0.041833333333333333, Train loss: 0.1299964866026285  |Validation err: 0.2445 , Validation loss:0.8536502728238702 
Epoch 6: Train err: 0.03383333333333333, Train loss: 0.09675123150202822  |Validation err: 0.2165 , Validation loss:0.8224746491760015 
Epoch 7: Train err: 0.028166666666666666, Train loss: 0.09213120129672771  |Validation err: 0.213 , Validation loss:0.8018942242488265 
Epoch 8: Train err: 0.024333333333333332, Train loss: 0.0686417687684297

In [24]:
train(netalex, batch_size = 32, learning_rate = 0.001, num_epochs = 15)

Epoch 1: Train err: 0.8636666666666667, Train loss: 6.696951688604152  |Validation err: 0.8235 , Validation loss:2.198826481425573 
Epoch 2: Train err: 0.7846666666666666, Train loss: 2.0924417465291123  |Validation err: 0.7975 , Validation loss:2.510080871127901 
Epoch 3: Train err: 0.7626666666666667, Train loss: 2.0622327657456094  |Validation err: 0.7625 , Validation loss:2.4051466385523477 
Epoch 4: Train err: 0.751, Train loss: 2.019769406065028  |Validation err: 0.7625 , Validation loss:2.2512343421814935 
Epoch 5: Train err: 0.7235, Train loss: 1.9731543495299968  |Validation err: 0.7465 , Validation loss:2.336838175380041 
Epoch 6: Train err: 0.6973333333333334, Train loss: 1.9247440936717581  |Validation err: 0.7205 , Validation loss:2.2657625448136103 
Epoch 7: Train err: 0.6743333333333333, Train loss: 1.8526354774515679  |Validation err: 0.6925 , Validation loss:2.011690378189087 
Epoch 8: Train err: 0.6598333333333334, Train loss: 1.827751729082554  |Validation err: 0.723

# Test Accuracy-Alex Net
Report the test accuracy of your best model. How does the test accuracy compare to part 4(d)?

In [48]:
def get_accuracyt(model):
    data = testset
    total = 0
    correct =0
    for imgs, labels in torch.utils.data.DataLoader(data, batch_size=32):
        #features = myfeature_model(imgs)
        output = model(imgs) # We don't need to run F.softmax
        pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += imgs.shape[0]
    return correct / total

In [49]:
net = AlexNet()
model_path = get_model_name(net.name, batch_size=32, learning_rate=0.0001, epoch=3)
state = torch.load(model_path)
net.load_state_dict(state)


In [50]:
len(testset)

2000

In [51]:
get_accuracyt(net)

0.794

In [ ]:
get_accuracy(net)

In [59]:
model_conv = torchvision.models.resnet18(pretrained = True)

In [60]:
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 10)

In [62]:
for name, child in model_conv.named_children():
    for name2, params in child.named_parameters():
        print(name, name2)

conv1 weight
bn1 weight
bn1 bias
layer1 0.conv1.weight
layer1 0.bn1.weight
layer1 0.bn1.bias
layer1 0.conv2.weight
layer1 0.bn2.weight
layer1 0.bn2.bias
layer1 1.conv1.weight
layer1 1.bn1.weight
layer1 1.bn1.bias
layer1 1.conv2.weight
layer1 1.bn2.weight
layer1 1.bn2.bias
layer2 0.conv1.weight
layer2 0.bn1.weight
layer2 0.bn1.bias
layer2 0.conv2.weight
layer2 0.bn2.weight
layer2 0.bn2.bias
layer2 0.downsample.0.weight
layer2 0.downsample.1.weight
layer2 0.downsample.1.bias
layer2 1.conv1.weight
layer2 1.bn1.weight
layer2 1.bn1.bias
layer2 1.conv2.weight
layer2 1.bn2.weight
layer2 1.bn2.bias
layer3 0.conv1.weight
layer3 0.bn1.weight
layer3 0.bn1.bias
layer3 0.conv2.weight
layer3 0.bn2.weight
layer3 0.bn2.bias
layer3 0.downsample.0.weight
layer3 0.downsample.1.weight
layer3 0.downsample.1.bias
layer3 1.conv1.weight
layer3 1.bn1.weight
layer3 1.bn1.bias
layer3 1.conv2.weight
layer3 1.bn2.weight
layer3 1.bn2.bias
layer4 0.conv1.weight
layer4 0.bn1.weight
layer4 0.bn1.bias
layer4 0.conv2.we

In [63]:
model_conv

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co